In [ ]:
!pip install openai==0.28
!pip install psycopg2
!pip install gradio

In [ ]:
import openai
import psycopg2
import gradio as gr

openai.api_key = 'sk-HTPT57K***************************************'

conn_params = {
    'dbname': 'reviews',
    'user': 'postgres',
    'password': 'admin',
    'host': 'localhost',
    'port': '5432'
}

def openai_create(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {"role": "system", "content": str("As a product feedback collecting person for an e-commerce platform, your role involves gathering user feedback and reviews to enhance our products' performance and user experience and  make sure you dont say any product. Your primary objective is to ensure that customers are providing review and feedback and respond accordingly. Please gather and filter any user-provided feedback. Initially ask like what product, then what brand, then what model, then review,rating, then ask are you satisfied with it and then any suggestions for improvement and so on.") + prompt},
        ],
        temperature=0.9,
        max_tokens=150,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0.6,
        stop=[" Human:", " AI:"]
    )
    return response.choices[0].message['content']

def openai_review(history):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {"role": "system", "content":"Please provide the following information separated by commas: Product, Brand, Review, Rating as a python list from"+history },
        ],
        temperature=0.9,
        max_tokens=150,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0.6,
        
    )
    info= response.choices[0].message['content']
    items = info.split(',')
    product = items[0].strip()[1:-1]
    brand = items[1].strip()[1:-1]
    review = ','.join(items[2:-1]).strip()[1:-1]
    rating = float(items[-1].strip()[:-1])
    info = [product, brand, review, rating]

    return info

def conversation_history(input, history):
    if input.lower() == 'exit':
        l = []
        for i in history:
            l.append(i[0])
        l = ",".join(l)
        review = openai_review(l)
        if review:
            upload_review_to_database([review])
        return None, None
    else:
        history = history or []
        s = list(sum(history, ()))
        s.append(input)
        inp = ' '.join(s)
        output = openai_create(inp)
        history.append((input, output))
        return history, history

def extract_review(history):
    reviews = []
    for user_input, ai_response in history:
        # Split AI response by 'Human' and take the last part which contains the review
        ai_parts = ai_response.split("Human:")
        if len(ai_parts) > 1:
            review = ai_parts[-1].strip()
            if review:
                reviews.append(review)
    return reviews

def upload_review_to_database(reviews):
    connection = psycopg2.connect(**conn_params)
    cursor = connection.cursor()
    for review in reviews:
        cursor.execute("INSERT INTO reviews (product, brand, review, rating) VALUES (%s,%s,%s,%s)", (review[0],review[1],review[2],review[3]))
    connection.commit()
    print("Reviews uploaded to the database successfully.")
    if connection:
        cursor.close()
        connection.close()
blocks = gr.Blocks()

with blocks:
    chatbot = gr.Chatbot()
    message = gr.Textbox(placeholder="Type here....")
    state = gr.State()
    submit = gr.Button("Click")
    submit.click(conversation_history, inputs=[message, state], outputs=[chatbot, state])

blocks.launch(debug=True) 
